In [10]:
pip install pydantic matplotlib numpy --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import pydantic
import json


In [2]:
load_dotenv()
client = OpenAI()

In [6]:
Model = "gpt-4.1-mini"

## 1. Basic Text Generation

In [12]:
prompt = "generate a single line story"

response = client.responses.create(
    model = Model,
    input = prompt,
    top_p = 0.9,
    temperature = 0.7
)

print("Single line story is: ", response.output_text)

Single line story is:  Beneath the fading stars, she whispered goodbye to the past and stepped into the dawn of her new life.


## 2. Getting Structured Outputs

To get structured output you can use pydantic. 
The response is generated using .parse instead of .create


### Using Chat Completions

In [14]:
Model = "gpt-4.1-mini"

In [26]:
from pydantic import BaseModel

class Step(BaseModel):
    explaination: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str 

input1  =[
    {
        "role":"system",
        "content": "You are an expert math tutor and you will guide the user step by step"
    },
    {
        "role":"user",
        "content":"How can I solve 4 + (3 -2) * 10 - 9 + 8"
    }
]



In [31]:
response = client.beta.chat.completions.parse(
    model = Model,
    messages = input1,
    response_format = MathReasoning,
)



In [32]:
print(response.choices[0].message)

ParsedChatCompletionMessage[MathReasoning](content='{"steps":[{"explaination":"First, solve the operation inside the parentheses: (3 - 2).","output":"3 - 2 = 1"},{"explaination":"Now replace the parentheses with the result, the expression becomes 4 + 1 * 10 - 9 + 8.","output":"4 + 1 * 10 - 9 + 8"},{"explaination":"Next, perform the multiplication before addition and subtraction according to the order of operations.","output":"1 * 10 = 10"},{"explaination":"The expression now is 4 + 10 - 9 + 8. Perform addition and subtraction from left to right.","output":"4 + 10 = 14"},{"explaination":"Continue with the next operation: 14 - 9.","output":"14 - 9 = 5"},{"explaination":"Finally, add the last number: 5 + 8.","output":"5 + 8 = 13"}],"final_answer":"13"}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=MathReasoning(steps=[Step(explaination='First, solve the operation inside the parentheses: (3 - 2).', output='3 - 2 = 1'), Step(explai

### Responses API

In [29]:
response1 = client.responses.parse(
    model = Model,
    input = input1,
    text_format = MathReasoning,
)

print(response1.output_text)

{"steps":[{"explaination":"First, solve the expression inside the parentheses: (3 - 2) = 1.","output":"4 + (1) * 10 - 9 + 8"},{"explaination":"Next, multiply the result from the parentheses by 10: 1 * 10 = 10.","output":"4 + 10 - 9 + 8"},{"explaination":"Now, perform addition and subtraction from left to right: 4 + 10 = 14","output":"14 - 9 + 8"},{"explaination":"Continue with subtraction: 14 - 9 = 5.","output":"5 + 8"},{"explaination":"Finally, add 5 + 8 = 13.","output":"13"}],"final_answer":"13"}


## 3. Multimodal Capabilities

Skipped right now (vision, text to speech, speech to text, )

## 4. Function Calling (Tool calling)

In [3]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']



tools = [{
    "type":"function",
    "name":"get_weather",
    "description":"Get current temperature for provided coordinates in degree celsius",
    "parameters": {
        "type":"object",
        "properties":{
            "latitude":{"type": "number"},
            "longitude":{"type":"number"}
        },
        "required":["latitude","longitude"],
        "additionalProperties": False 
    },
    "strict":True
}]

In [16]:
input_messages = [{"role":"user", "content":"What is the weather like in Delhi Today?"}]

response = client.responses.create(
    model = "gpt-4o",
    input = input_messages,
    tools = tools,
)

In [17]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args['latitude'], args['longitude'])

In [18]:
# Appending the tool call and result to the input messages
input_messages.append(tool_call)
input_messages.append({
    "type":"function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model = "gpt-4o",
    input = input_messages,
    tools = tools,

)
print(response_2.output_text)


The current temperature in Delhi is 26.8°C.


## 5. Reasoning Model

Current Plan does not support Reasoning

In [ ]:
response_o4 = client.responses.create(
    model = "o4-mini",
    input = "Explain how to implement a Machine Learning Project?"
)

print(response_o4.output_text)

## 6. Embeddings


In [20]:
import numpy as np

In [27]:
words = [
    "Pranav", "King", "Mulitmodal",
    "AI", "Ambition", "Queen"
]

response = client.embeddings.create(
    model = "text-embedding-3-large",
    input = words,
    encoding_format = "float"
)
